# Exploratory Notebook


*This notebook has been made by Martin Sejas*

### Setup - Importing packages

In [16]:
import pandas as pd
import numpy as np
import sklearn 
import typing

## Step 1: Loading and Splitting Dataset 

In [17]:
#loading csv file
master = pd.read_csv("../raw_data/flight-price-training.csv")

#creating a copy for exploration
df = master.copy()

#Dropping the first column that is simply the original index of row before partitioning
df = df.drop(columns=df.columns[0])

print(df.shape)

df.head()


(240122, 11)


,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,Air_India,AI-804,Bangalore,Early_Morning,one,Night,Mumbai,Business,16.00,11,54684
1,Air_India,AI-503,Bangalore,Evening,one,Night,Hyderabad,Economy,6.25,12,13054
2,GO_FIRST,G8-426,Hyderabad,Night,one,Morning,Bangalore,Economy,10.33,34,6256
3,Indigo,6E-534,Kolkata,Evening,one,Night,Chennai,Economy,5.33,21,5280
4,Vistara,UK-863,Mumbai,Morning,two_or_more,Night,Chennai,Economy,11.17,45,8130


We can see here that we have 10 features and 1 label ('price'), we can also see a lot of our features are categorical, and encoded in an inconvenient manner, for example, the column ('stops') is in string format instead of number, we will have to see the different classes contained in this column to decide if we convert it to a number or keep it as a categorical variable. Similar treatment will be needed for other variables, such as 'departure_time' decide how we treat them. 



Lets check for any nan's, and see general info of our dataset.


In [18]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240122 entries, 0 to 240121
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   airline           240122 non-null  object 
 1   flight            240122 non-null  object 
 2   source_city       240122 non-null  object 
 3   departure_time    240122 non-null  object 
 4   stops             240122 non-null  object 
 5   arrival_time      240122 non-null  object 
 6   destination_city  240122 non-null  object 
 7   class             240122 non-null  object 
 8   duration          240122 non-null  float64
 9   days_left         240122 non-null  int64  
 10  price             240122 non-null  int64  
dtypes: float64(1), int64(2), object(8)
memory usage: 20.2+ MB


As we can see, all of the features + label match the shape of the dataframe, 240122, meaning that there are no NaN values so no imputation technique would be needed. We can proceed to check the different classes in our features.

In [19]:
for col in df.columns:
    print(f"{col}: Unique values: {df[col].unique()}")
    print(f"count: {len(df[col].unique())}")
    print()

airline: Unique values: ['Air_India' 'GO_FIRST' 'Indigo' 'Vistara' 'AirAsia' 'SpiceJet']
count: 6

flight: Unique values: ['AI-804' 'AI-503' 'G8-426' ... '6E-543' '6E-865' 'I5-1427']
count: 1556

source_city: Unique values: ['Bangalore' 'Hyderabad' 'Kolkata' 'Mumbai' 'Delhi' 'Chennai']
count: 6

departure_time: Unique values: ['Early_Morning' 'Evening' 'Night' 'Morning' 'Afternoon' 'Late_Night']
count: 6

stops: Unique values: ['one' 'two_or_more' 'zero']
count: 3

arrival_time: Unique values: ['Night' 'Morning' 'Afternoon' 'Early_Morning' 'Evening' 'Late_Night']
count: 6

destination_city: Unique values: ['Mumbai' 'Hyderabad' 'Bangalore' 'Chennai' 'Delhi' 'Kolkata']
count: 6

class: Unique values: ['Business' 'Economy']
count: 2

duration: Unique values: [16.    6.25 10.33  5.33 11.17  9.75  2.17 10.5   8.83 15.17  8.75  2.
 16.08  2.92  3.58  6.75  9.5   8.08 22.5   4.42 14.58 11.25 11.75 21.5
  5.83 11.    9.25  6.17  8.   12.25 35.75  6.92  6.83 22.58  9.83  3.92
  8.25 18.17 16.5 

We can see we will have a feature implosion with the flights, as there are 1556 unique values, additionally the actual code of the flight should not affect the price of the model. Considering I don't have many features, I will make two models, one with it, and one without.

### Splitting the dataset

I will split the dataset into 80% training data, 10% validation, and 10% testing data. I will stratify the 'class' of the ticket (Business/Economy) to make sure my data is well balanced.

In [20]:
from sklearn.model_selection import train_test_split 

#separating features from label
Y = df['price']
X = df.drop(columns=['price'])

#Setting 80% to the training set
X_train, X_rest, y_train, y_rest = train_test_split(X, Y, test_size=0.2, stratify=X['class'], random_state=42)

#Setting 10% to the validation set and 10% to the test set
X_valid, X_test, y_valid, y_test = train_test_split(X_rest, y_rest, test_size=0.5,stratify=X_rest['class'], random_state=42 )



## Step 2: Preprocessing

As mentioned before, the 'stops' feature needs to be ordinally encoded. The functions below take care of this.  

In [23]:
#This function will read a string with 'one' ,'two_or_more' or 'zero' and return the appropriate integer
def encode_stops(s:str)-> int:
    if (s == 'one'):
        return 1
    elif(s == 'zero'):
        return 0
    else:
        return 2
    
def process_stops_column(df: pd.DataFrame)->pd.DataFrame:
    df['stops'] = df['stops'].apply(encode_stops)
    return df

In [26]:
#testing our function 

df_test = df.copy()

df_test_processed = process_stops_column(df=df_test)

df_test_processed.head()

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,Air_India,AI-804,Bangalore,Early_Morning,1,Night,Mumbai,Business,16.00,11,54684
1,Air_India,AI-503,Bangalore,Evening,1,Night,Hyderabad,Economy,6.25,12,13054
2,GO_FIRST,G8-426,Hyderabad,Night,1,Morning,Bangalore,Economy,10.33,34,6256
3,Indigo,6E-534,Kolkata,Evening,1,Night,Chennai,Economy,5.33,21,5280
4,Vistara,UK-863,Mumbai,Morning,2,Night,Chennai,Economy,11.17,45,8130


Hence now we can implement our main pre-processing steps.

In [40]:
from sklearn.preprocessing import (StandardScaler, OneHotEncoder)
from sklearn.compose import ColumnTransformer

#Here I want to make two separate column transformers, one with the 'flight' column, and one without. 
#Hence I will here make my distinctions

X_train_no_flight = X_train.drop(columns=['flight'])
X_valid_no_flight = X_valid.drop(columns=['flight'])
X_test_no_flight = X_test.drop(columns=['flight'])

#dropping the 'stops' feature because this will be handled differently
#Applying initial custom transformation
X_train_p_stops = process_stops_column(X_train)
X_valid_p_stops = process_stops_column(X_valid)
X_test_p_stops = process_stops_column(X_test)

X_train_no_flight_p_stops = process_stops_column(X_train_no_flight)
X_valid_no_flight_p_stops = process_stops_column(X_valid_no_flight)
X_test_no_flight_p_stops = process_stops_column(X_test_no_flight)


#getting the right categorical columns 
full_categorical_columns = (X_train_p_stops.select_dtypes(include=['object'])).columns
partial_categorical_columns = (X_train_no_flight_p_stops.select_dtypes(include=['object'])).columns

#Extracting numerical columns
numerical_columns = list(set(X_train_p_stops.columns) - set(full_categorical_columns))

#transformer with 'flight' column included
ct_main = ColumnTransformer(
                [("ohe", OneHotEncoder(drop='if_binary', sparse_output=False, handle_unknown='ignore'), full_categorical_columns),
                 ("std", StandardScaler(), numerical_columns)
                ])

#transformer without 'flight' column included
ct_secondary = ColumnTransformer(
                [("ohe", OneHotEncoder(drop='if_binary', sparse_output=False, handle_unknown='ignore'), partial_categorical_columns),
                 ("std", StandardScaler(), numerical_columns)
                ])

#Applying main pre-processing
ct_main.fit(X_train_p_stops)
X_train_p = ct_main.transform(X_train_p_stops)
X_valid_p = ct_main.transform(X_valid_p_stops)
X_test_p = ct_main.transform(X_test_p_stops)

#Applying secondary pre-processing
ct_secondary.fit(X_train_no_flight_p_stops)
X_train_no_flight_p = ct_secondary.transform(X_train_no_flight_p_stops)
X_valid_no_flight_p = ct_secondary.transform(X_valid_no_flight_p_stops)
X_test_no_flight_p = ct_secondary.transform(X_test_no_flight_p_stops)

#Label is numerical, so we need to also normalize it
y_std = StandardScaler()

#Reshaping y to pre-process
y_train = (y_train.to_numpy()).reshape(-1,1)
y_valid = (y_valid.to_numpy()).reshape(-1,1)
y_test = (y_test.to_numpy()).reshape(-1,1)

#Pre-processing the label
y_std.fit(y_train)
y_train_p = y_std.transform(y_train)
y_valid_p = y_std.transform(y_valid)
y_test_p = y_std.transform(y_test)

d:\anaconda3\envs\FlightPricePredictionEnv\Lib\site-packages\sklearn\preprocessing\_encoders.py:202: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
d:\anaconda3\envs\FlightPricePredictionEnv\Lib\site-packages\sklearn\preprocessing\_encoders.py:202: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


## Step 3: Training and Evaluating Models

For the very first model we will just use a simple Linear Regression (Gradient Descent) and our main initial goal here, is to see if there are any impactful performance differences in having the flight code or not.

In [41]:
from sklearn.linear_model import LinearRegression

#with flight column
main_model = LinearRegression()

#fitting the main model
main_model.fit(X_train_p,y=y_train_p)

main_model.score(X=X_test_p, y=y_test_p)



-3681745022692510.0

In [42]:
#Without flight column
model_no_flights = LinearRegression()

model_no_flights.fit(X=X_train_no_flight_p, y=y_train_p)

model_no_flights.score(X=X_test_no_flight_p, y=y_test_p)

0.9005264698784824

The 'score' method called in each method calculates how well the predictions correlate with the validation set. It's called the coefficient of determination, where the best 'score' is 1.0. We can see here that the model that included the flight-codes is garbage, while the model that excludes the 'flights' column has a very good score of 0.9.

Having that in mind we will explore future models excluding the 'flights' column. 

Calculating the mean squared error (for no_flights model):

In [92]:
#checking the error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error

y_preds = model_no_flights.predict(X_valid_no_flight_p)
mse_valid = mean_squared_error(y_true=y_valid_p,y_pred=y_preds )
print(f"MSE valid: {mse_valid}")

y_preds = model_no_flights.predict(X_test_no_flight_p)
mse_test = mean_squared_error(y_true=y_test_p,y_pred=y_preds )
print(f"MSE test: {mse_test}")

print(f"Average MSE: {sum([mse_valid,mse_test])/2}")

masp = mean_absolute_percentage_error(y_true = y_test, y_pred=(y_std.inverse_transform(y_preds)))
print(f"Mean Absolute Percentage Error: {masp*100}%")



MSE valid: 0.09688468443307602
MSE test: 0.10007568539728301
Average MSE: 0.09848018491517951
Mean Absolute Percentage Error: 36.7803999879977%


We can also use the mean absolute percentage error to have an idea of the performance of our models.

**NOTE: The MSE's have been calculated on the normalized predictions, and validation sets.**

**NOTE: The masp has been calculated on values of the actual dataset.**

Note that this is done on the actual flight prices in (indian rupees)

We can see that our Mean Absolute Percentage Error (36%) seems high. However we must consider we only have around 7 features. If we had more features, for example date, where price hikes due to holidays could be factored in, we would have a more performing model.



As we can see the model without the flights column is superior, and converges much faster. We will attempt to achieve a better performance by applying regularization on our model.

In [53]:
from sklearn.linear_model import Ridge


#generating a range of alphas 
alphas = np.logspace(-5,5,100)

best_alpha = alphas[0]
best_score = -1000000

for alpha in alphas:
    
    model_ridge = Ridge(alpha=alpha)

    model_ridge.fit(X_train_no_flight_p, y_train_p)
    score = model_ridge.score(X=X_test_no_flight_p, y=y_test_p)
    
    if(score > best_score):
        best_score = score
        best_alpha = alpha


print(best_score)
print(best_alpha)


0.9005309703283476
1e-05


In [84]:
#checking the error

y_preds = model_ridge.predict(X_valid_no_flight_p)
mse_valid = mean_squared_error(y_true=y_valid_p,y_pred=y_preds )
print(f"MSE valid: {mse_valid}")

y_preds = model_ridge.predict(X_test_no_flight_p)
mse_test = mean_squared_error(y_true=y_test_p,y_pred=y_preds )
print(f"MSE test: {mse_test}")

print(f"Average MSE: {sum([mse_valid,mse_test])/2}")
masp = mean_absolute_percentage_error(y_true = y_test, y_pred=y_std.inverse_transform(y_preds))
print(f"Mean Absolute Percentage Error: {masp*100}%")



MSE valid: 0.4950850965963323
MSE test: 0.5041028720954136
Average MSE: 0.499593984345873
Mean Absolute Percentage Error: 150.95730624697092%


It seems that implement Ridge regularization did show improvement to our model, as we have a lower Mean Squared Error(MSE). 

Having that in mind we will check if an elastic net with cross validation would improve our model further.

In [85]:
from sklearn.linear_model import ElasticNetCV


model_en = ElasticNetCV(random_state=42)

#fitting the main model
model_en.fit(X=X_train_no_flight_p, y=y_train_p.ravel())

print(f"best alpha = {model_en.alpha_}")

model_en.score(X=X_test_no_flight_p, y=y_test_p)



best alpha = 0.0008689976557719299


0.9004640723995209

Calculating MSE for our ElasticNet model

In [93]:
y_preds = model_en.predict(X_valid_no_flight_p)
mse_valid = mean_squared_error(y_true=y_valid_p,y_pred=y_preds )
print(f"MSE valid: {mse_valid}")

y_preds = model_en.predict(X_test_no_flight_p)
mse_test = mean_squared_error(y_true=y_test_p,y_pred=y_preds )
print(f"MSE test: {mse_test}")

print(f"Average MSE: {sum([mse_valid,mse_test])/2}")

masp = mean_absolute_percentage_error(y_true = y_test, y_pred=y_std.inverse_transform(y_preds.reshape(-1,1)))
print(f"Mean Absolute Percentage Error: {masp*100}%")



MSE valid: 0.09690949926382428
MSE test: 0.10013846059452904
Average MSE: 0.09852397992917666
Mean Absolute Percentage Error: 36.30954982794132%


Here we can see that the ElasticNet Model which implements L1 and L2 regularization + GridSearch model greatly outperforms the standard LinearRegressor and RidgeRegressor. 

With an average MSE of 0.0985 the results are excellent.



 Here we will try to use a Polynomial Transformation to see if we can get a better result

In [78]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(2)

poly.fit(X_train_no_flight_p)

model_en_poly = ElasticNetCV(random_state=42)


#fitting the main model
model_en_poly.fit(X=poly.transform(X_train_no_flight_p), y=y_train_p.ravel())

print(f"best alpha = {model_en_poly.alpha_}")

model_en_poly.score(X=poly.transform(X_test_no_flight_p), y=y_test_p)

d:\anaconda3\envs\FlightPricePredictionEnv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.841e+01, tolerance: 1.921e+01
  model = cd_fast.enet_coordinate_descent(


best alpha = 0.0008689976557719299


0.9310980323760176

Calculating Metrics

In [94]:
y_preds = model_en_poly.predict(poly.transform(X_valid_no_flight_p))
mse_valid = mean_squared_error(y_true=y_valid_p,y_pred=y_preds )
print(f"MSE valid: {mse_valid}")

y_preds = model_en_poly.predict(poly.transform(X_test_no_flight_p))
mse_test = mean_squared_error(y_true=y_test_p,y_pred=y_preds )
print(f"MSE test: {mse_test}")

print(f"Average MSE: {sum([mse_valid,mse_test])/2}")

masp = mean_absolute_percentage_error(y_true = y_test, y_pred=y_std.inverse_transform(y_preds.reshape(-1,1)))
print(f"Mean Absolute Percentage Error: {masp*100}%")

MSE valid: 0.06695864048268976
MSE test: 0.06931906032457026
Average MSE: 0.06813885040363002
Mean Absolute Percentage Error: 33.26128055926997%


## Step 4: Conclusion and Final Thoughts